### Imports

In [ ]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import get_dataset_class
from resnet.plot import plot_domainwise_accuracy
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

DS = "VLCS"
MODEL_NAME = "resnet18"

Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [ ]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

DatasetClass = get_dataset_class(DS)

ld = DatasetClass(
    data_root=CFG["datasets"][DS]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [ ]:
dataset_name = DS
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-10-10 07:23:05,174 | INFO | Initialized experiment directories for VLCS
2025-10-10 07:23:05,174 | INFO | Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\VLCS\logs
2025-10-10 07:23:05,174 | INFO | Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\VLCS\checkpoints
2025-10-10 07:23:05,174 | INFO | Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\VLCS\plots


### Setup

In [ ]:
domains = CFG["datasets"][DS]["domains"]
loaders = {d: {"train": ld.get_dataloader(d, train=True), "val": ld.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset=DS)
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]


{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [ ]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

2025-10-10 07:23:05,473 | INFO | === LODO: Leaving out domain 'VOC2007' ===



=== LODO: Leaving out domain 'VOC2007' ===


Evaluating: 100%|██████████| 27/27 [00:13<00:00,  2.07it/s]
2025-10-10 07:23:49,470 | INFO | [VOC2007] Epoch 1/5 | Train - Loss: 0.6993, Cls: 0.6900, GRQO: 0.0093, Acc: 0.7242 | Val - Loss: 0.7433, Cls: 0.7421, GRQO: 0.0012, Acc: 0.7171
2025-10-10 07:23:49,543 | INFO | [VOC2007] New best val acc: 0.7171


[VOC2007] Epoch 1/5 | Train - Loss: 0.6993, Cls: 0.6900, GRQO: 0.0093, Acc: 0.7242 | Val - Loss: 0.7433, Cls: 0.7421, GRQO: 0.0012, Acc: 0.7171
[VOC2007] New best val acc: 0.7171


Evaluating: 100%|██████████| 27/27 [00:12<00:00,  2.15it/s]
2025-10-10 07:24:31,971 | INFO | [VOC2007] Epoch 2/5 | Train - Loss: 0.3640, Cls: 0.3615, GRQO: 0.0025, Acc: 0.8681 | Val - Loss: 0.7022, Cls: 0.7014, GRQO: 0.0009, Acc: 0.7482
2025-10-10 07:24:32,083 | INFO | [VOC2007] New best val acc: 0.7482


...

[SUN09] Epoch 3/5 | Train - Loss: 0.1689, Cls: 0.1668, GRQO: 0.0021, Acc: 0.9421 | Val - Loss: 0.9452, Cls: 0.9440, GRQO: 0.0012, Acc: 0.7127
[SUN09] New best val acc: 0.7127


Evaluating: 100%|██████████| 26/26 [00:36<00:00,  1.39s/it]
2025-10-10 08:01:29,533 | INFO | [SUN09] Epoch 4/5 | Train - Loss: 0.0539, Cls: 0.0520, GRQO: 0.0020, Acc: 0.9831 | Val - Loss: 1.2495, Cls: 1.2488, GRQO: 0.0007, Acc: 0.6801


[SUN09] Epoch 4/5 | Train - Loss: 0.0539, Cls: 0.0520, GRQO: 0.0020, Acc: 0.9831 | Val - Loss: 1.2495, Cls: 1.2488, GRQO: 0.0007, Acc: 0.6801


Evaluating: 100%|██████████| 26/26 [00:36<00:00,  1.40s/it]
2025-10-10 08:04:15,318 | INFO | [SUN09] Epoch 5/5 | Train - Loss: 0.0231, Cls: 0.0213, GRQO: 0.0018, Acc: 0.9936 | Val - Loss: 1.5497, Cls: 1.5493, GRQO: 0.0005, Acc: 0.6843
2025-10-10 08:04:15,318 | INFO | [SUN09] Best Acc: 0.7127
2025-10-10 08:04:15,318 | INFO | ------------------------------------------------------------
2025-10-10 08:04:15,318 | INFO | LODO finished | Mean Acc: 0.7753 | Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\VLCS\logs\lodo_summary_20251010_080415.json


[SUN09] Epoch 5/5 | Train - Loss: 0.0231, Cls: 0.0213, GRQO: 0.0018, Acc: 0.9936 | Val - Loss: 1.5497, Cls: 1.5493, GRQO: 0.0005, Acc: 0.6843
[SUN09] Best Acc: 0.7127
------------------------------------------------------------
LODO finished | Mean Acc: 0.7753
Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\VLCS\logs\lodo_summary_20251010_080415.json


### Baseline

In [ ]:
baseline_results, baseline_mean = run_baseline(
    model_name=MODEL_NAME,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)

2025-10-10 08:04:15,336 | INFO | Initializing ResNet baseline: resnet18
2025-10-10 08:04:15,428 | INFO | === Baseline LODO: Leaving out domain 'VOC2007' ===


Initializing ResNet baseline: resnet18

=== Baseline LODO: Leaving out domain 'VOC2007' ===


2025-10-10 08:05:57,766 | INFO | [VOC2007] Epoch 1/5 | Train - Loss: 0.6753, Acc: 0.7489 | Val Acc: 0.7441


[VOC2007] Epoch 1/5 | Train - Loss: 0.6753, Acc: 0.7489 | Val Acc: 0.7441


2025-10-10 08:07:39,027 | INFO | [VOC2007] Epoch 2/5 | Train - Loss: 0.2813, Acc: 0.9056 | Val Acc: 0.7586


...

[SUN09] Epoch 3/5 | Train - Loss: 0.1170, Acc: 0.9694 | Val Acc: 0.6840


2025-10-10 08:38:38,726 | INFO | [SUN09] Epoch 4/5 | Train - Loss: 0.0353, Acc: 0.9946 | Val Acc: 0.7252


[SUN09] Epoch 4/5 | Train - Loss: 0.0353, Acc: 0.9946 | Val Acc: 0.7252


2025-10-10 08:40:16,789 | INFO | [SUN09] Epoch 5/5 | Train - Loss: 0.0116, Acc: 0.9995 | Val Acc: 0.7148
2025-10-10 08:40:16,790 | INFO | [SUN09] Best Val Acc: 0.7252
2025-10-10 08:40:16,790 | INFO | ------------------------------------------------------------
2025-10-10 08:40:16,791 | INFO | Baseline LODO (resnet18) finished | Mean Acc: 0.7817


[SUN09] Epoch 5/5 | Train - Loss: 0.0116, Acc: 0.9995 | Val Acc: 0.7148
[SUN09] Best Val Acc: 0.7252
------------------------------------------------------------
Baseline LODO (resnet18) finished | Mean Acc: 0.7817
